Часть материала украдена из курса "Глубинное обучение" ФКН ВШЭ https://www.hse.ru/ba/ami/courses/205504078.html, за что им большое спасибо.

# Устанавливаем pytorch

## Linux/OSX


На оффсайте http://pytorch.org/ надо выбрать подходящую конфигурацию и скачать.

Версию python можно узнать в терминале:
```
python --version
```


## Windows without GPU

Проще всего поставить при помощи конды:
```
conda install -c peterjc123 pytorch
```

## Windows with GPU

Смотрите https://github.com/peterjc123/pytorch-scripts

![img](https://s1.postimg.org/6fl45xnvnj/pytorch-logo-dark.png)

In [ ]:
import torch
import numpy as np

In [ ]:
# numpy world

x = np.arange(16).reshape(4, 4)

print("X :\n %s" % x)
print("add 5 :\n%s" % (x + 5))
print("X*X^T  :\n", np.dot(x, x.T))
print("mean over cols :\n%s" % (x.mean(axis=-1)))
print("cumsum of cols :\n%s" % (np.cumsum(x, axis=0)))

In [ ]:
# pytorch world

x = np.arange(16).reshape(4, 4)

y = torch.from_numpy(x)
print('Type of y:', type(y))
y = torch.from_numpy(x).type(torch.FloatTensor)
print('Type of y', type(y))

In [ ]:
x = torch.arange(0,16).view(4,4)
print('Type of x:', type(x))

print("X :\n%s" % x)
print("add 5 :\n%s" % (x + 5))
print("X*X^T  :\n", torch.matmul(x, x.transpose(1, 0)))
print("mean over cols :\n", torch.mean(x, dim=-1))
print("cumsum of cols :\n", torch.cumsum(x, dim=0))

## NumPy vs Pytorch

Numpy и Pytorch не требуют описания статического графа вычислений. 

Можно отлаживаться с помощью pdb или просто print.

API несколько различается:

```
x.reshape([1,2,8]) -> x.view(1,2,8)
x.sum(axis=-1) -> x.sum(dim=-1)
x.astype('int64') -> x.type(torch.LongTensor)
```


Легко конвертировать между собой:

```
torch.from_numpy(npx) -- вернет Tensor
tt.numpy() -- вернет Numpy Array
```


Если что:
- смотрите документацию
- гуглите (Stackoverflow/tutorials)

In [ ]:
x = torch.linspace(0, 2 * np.pi, 16)

# Mini-task: compute a vector of sin^2(x) + cos^2(x)
out = <your code>

print(out.numpy())

## In-place operations

Когда работаем с большими массивами, память надо экономить.
Некоторые операции происходят с созданием нового объекта - результата вычислений,
некоторые изменяют данный объект (in-place операции).
В pytorch обычно эти операции различаются добавлением подчеркивания:
```
x.exp()   # not-in-place operation
x.exp_()  # in-place operation
```

In [ ]:
x = torch.arange(4)
print('Not-in-place:')
print('\tx.exp():\t\t', x.exp().numpy())
print('\tx:\t\t\t', x.numpy())
print('In-place:')
print('\tx.exp_():\t\t', x.exp_().numpy())
print('\tx after x.exp_():\t', x.numpy())

In [ ]:
x = torch.arange(0, 4).view(2, 2)
y = torch.arange(4, 8).view(2, 2)
z = torch.arange(8, 12).view(2, 2)

# Not-in-place:
u = x + 2 * y - z    # 3 array allocations?
print(u.numpy())

# In-place
u = y.clone()        # 1 array allocation
u.mul_(2)
u.add_(x)
u.sub_(z)
print(u.numpy())

## Работа с тензорами

Для каждого объекта из X найти индекс ближайшего объекта из Y.

In [ ]:
X = torch.randn(100, 10)
Y = torch.randn(5, 10)

In [ ]:
<your code here>

## CUDA
`x.cuda()` копирует тензор на GPU и возвращает объект, соответствующий этому скопированному тензору.
Можно явно указать номер GPU, на который нужно скопировать тензор: `x.cuda(gpu_id)`.
Если тензор уже лежал на нужном GPU, то возвращается сам тензор, копирования не производится.
Аналогично работает `x.cpu()`. 

Операции можно осуществлять только над тензорами, лежащими на одном устройстве.
Нарушение этого правила приводит к ошибке.
Результат операции находится на том же устройстве, что и операнды.

# Tensor vs Variable

http://pytorch.org/docs/master/autograd.html#variable

`Variable` – обертка над Tensor для использования в вычислительных графах. Позволяет вычислять градиенты автоматически.

Tensor и Variable конвертируются друг в друга:
```
tensor to variable: Variable(x)
variable to tensor: x.data
```

Нельзя смешивать Tensor и Variable в одной операции.

Некоторые операции могут работать только с тензорами, некоторые только с переменными (torch.nn.functional.whatever).

In [ ]:
from torch.autograd import Variable

sequence = torch.randn(1, 8, 10)
filters = torch.randn(2, 8, 3)
a = torch.randn(5)
b = torch.randn(5)

In [ ]:
# works:
print('conv1d over Variables:')
print(torch.nn.functional.conv1d(Variable(sequence), Variable(filters)))

In [ ]:
# works:
print('sum of Variables:')
print(Variable(a) + Variable(b))

In [ ]:
# will not work
print("conv1d (tensors):")
print(torch.nn.functional.conv1d(sequence, filters))

In [ ]:
# will not work:
print('sum of Variable and Tensor:')
print(Variable(a) + b)

# Automatic gradients

Автоматическое вычисление градиентов:

1. Создать переменную: `a = Variable(..., requires_grad=True)`

2. Определить какую-нибудь дифференцируемую функцию `loss = whatever(a)`

3. Запросить обратный проход `loss.backward()`

4. Градиенты будут доступны в `a.grads`


Есть два важных отличия Pytorch от Theano/TF:

1. Функцию ошибки можно изменять динамически, например на каждом минибатче.

2. После вычисления `.backward()` градиенты сохраняются в `.grad` каждой задействованной переменной, при повторных вызовах градиенты суммируются. Это позволяет использовать несколько функций ошибок или виртуально увеличивать batch_size. Поэтому после каждого шага оптимизатора градиенты стоит обнулять.

## Leaf vs Non-leaf Variable

Градиенты будут сохранены и доступны для использования только для `leaf-variable`.
Такое поведение по умолчанию сделано ради экономии памяти.

In [ ]:
x = Variable(torch.randn(4), requires_grad=True)  # leaf variable
y = x + 1  # not a leaf variable
y.sum().backward()
print('x.grad:', x.grad.data.numpy(), '\ny.grad:', y.grad)

## Листовые вершины без градиентов
Листовые вершины, в которых не требуется вычислять градиент, создаются с помощью `Variable(..., requires_grad=False)`.
Для корректного вызова `.backward()` требуется, чтобы хотя бы для одной листовой вершины требовался градиент.

In [ ]:
x = Variable(torch.randn(4), requires_grad=False)  # leaf variable
z = Variable(torch.randn(4), requires_grad=True)   # leaf variable
y = x + z + 1  # not a leaf variable
y.sum().backward()
print('x.grad:', x.grad,
      '\nz.grad:', z.grad.data.numpy(),
      '\ny.grad:', y.grad)

In [ ]:
# will not work:
x = Variable(torch.randn(4), requires_grad=False)   # leaf variable
z = Variable(torch.randn(4), requires_grad=False)   # leaf variable
y = x + z + 1  # not a leaf variable
y.sum().backward()
print('x.grad:', x.grad,
      '\nz.grad:', z.grad.data.numpy(),
      '\ny.grad:', y.grad)

## Градиенты промежуточных вершин
Для промежуточных вершин мы можем запросить сохранение градиентов с помощью функции `.retain_grad()`.

In [ ]:
x = Variable(torch.randn(4), requires_grad=True)   # leaf variable
z = Variable(torch.randn(4), requires_grad=True)   # leaf variable
w = z * 2      # not a leaf variable
y = x * w + 1  # forward pass before retaining gradient is ok

w.retain_grad()

y.sum().backward()
print('d y_sum / dx:', x.grad.data.numpy(),
      '\nd y_sum / dw:', w.grad.data.numpy(),
      '\nd y_sum / dz:', z.grad.data.numpy())

Обратим внимание, что даже при наличии в графе вычислений не-листовых вершин, требующих вычисления градиентов,
`.backward()` выдает ошибку.

In [ ]:
x = Variable(torch.randn(4), requires_grad=False)   # leaf variable
z = Variable(torch.randn(4), requires_grad=False)   # leaf variable
w = z * 2      # not a leaf variable
y = x * w + 1  # forward pass before retaining gradient is ok

w.retain_grad()

y.sum().backward()
print('d y_sum / dx:', x.grad.data.numpy(),
      '\nd y_sum / dw:', w.grad.data.numpy(),
      '\nd y_sum / dz:', z.grad.data.numpy())

## Отстреливаем себе ноги

Конвертировать Variable в Tensor и обратно.

In [ ]:
# x out of the computational graph
x = torch.autograd.Variable(torch.Tensor([1, 2, 3, 4]), requires_grad=True)
z = torch.autograd.Variable(x.data * 2, requires_grad=True)
y = 3 * z + 1
y_mean = y.mean()
y_mean.backward()
print('d y_mean / dx:', x.grad, '\nd y_sum / dz:', z.grad.data.numpy())

Менять значения в Variable, но не обнулять градиенты (.grad.zero_() сохраняет размер, .grad = None не сохраняет).

In [ ]:
x = torch.autograd.Variable(torch.Tensor([1, 2, 3, 4]), requires_grad=True)
z = torch.autograd.Variable(torch.Tensor([1, 2, 3, 4]), requires_grad=True)
y = x * z + 1
y.sum().backward()
print('d y_sum / dx:', x.grad.data.numpy(), '\nd y_sum / dz:', z.grad.data.numpy())

#x.grad.zero_()
#z.grad.zero_()
x.grad = None
z.grad = None

z.data = torch.Tensor([1, 2, 3])
x.data = torch.Tensor([1, 2, 3])
y = x * z + 1
y.sum().backward()
print('d y_sum / dx:', x.grad.data.numpy(), '\nd y_sum / dz:', z.grad.data.numpy())

Менять значения Variable после вычисления каких-то других выражений с ним и рассчитывать,
что градиент от тех выражений будет учитывать новое значение.

In [ ]:
x = torch.autograd.Variable(torch.Tensor([1, 2, 3, 4]), requires_grad=True)
z = torch.autograd.Variable(torch.Tensor([1, 2, 3, 4]), requires_grad=True)
w = z ** 2

w.data = torch.Tensor([1, 2, 3, 4])  # changing .data before computation matters
y = x * w + 1
x.data = torch.Tensor([1, 1, 1])     # changing .data after computation doesn't affect gradients

y.sum().backward()
print('d y_sum / dx:', x.grad.data.numpy(), '\nd y_sum / dz:', z.grad.data.numpy())

Тысячи их!

## Пример задачи

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
X = torch.randn(50, 10)
b = torch.randn(2)
W = torch.randn(10, 2)

Наркоманская функция потерь, вынуждающая линейное преобразование переводить точки из многомерного пространства в двумерное на единичную окружность. Для оптимизации использовать градиентный спуск.
$$y = Wx + b$$
$$||y||_2 = \sqrt{y_1^2 + y_2^2}$$
$$f(x, W, b) = \big| ||y||_2 - 1 \big| + max\big(0, \frac{1}{||y - b||_2} - 1\big)$$
$$\frac{1}{N}\sum\limits_{i = 1}^N f(x_i, W, b) \to \min\limits_{W, b}$$

In [ ]:
def f(X, W, b):
    <your code here>

In [ ]:
print(f(X, W, b))

In [ ]:
plt.figure(figsize=(5, 5))
Y = X.mm(W).add(b)
plt.scatter(Y[:, 0], Y[:, 1])

In [ ]:
<your code here>

In [ ]:
plt.figure(figsize=(5, 5))
Y = X.mm(W).add(b)
plt.scatter(Y[:, 0], Y[:, 1])

# Время писать нейросеть

## Загружаем данные

In [ ]:
from sklearn.datasets import load_digits

X, y = load_digits(return_X_y=True)
X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train.shape, X_test.shape

In [ ]:
X_train = torch.from_numpy(X_train).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train).type(torch.LongTensor)
X_test = torch.from_numpy(X_test).type(torch.FloatTensor)
y_test = torch.from_numpy(y_test).type(torch.LongTensor)

В ячейке ниже чего-то не хватает.

In [ ]:
<your code here>

## Определяем слои нейросети

In [ ]:
class Dense:
    def __init__(self, input_units, output_units):
        """
        A dense layer is a layer which performs a learned affine transformation:
        f(x) = W x + b
        """
        # initialize weights with small random numbers from normal distribution
        self.weights = <your code here>
        self.biases = <your code here>
        self.params = [self.weights, self.biases]
        
    def forward(self, input):
        """
        Perform an affine transformation:
        f(x) = W x + b
        
        input shape: [batch, input_units]
        output shape: [batch, output units]
        """
        <your code here>
        return output

    def train(self):
        """
        Set layer into train mode (for DropOut, BatchNorm, etc)
        """
        pass

    def eval(self):
        """
        Set layer into evaluation mode
        """
        pass

In [ ]:
class ReLU:
    def __init__(self):
        """
        ReLU layer simply applies elementwise rectified linear unit to all inputs
        """
        self.params = [] # ReLU has no parameters
    
    def forward(self, input):
        """
        Apply elementwise ReLU to [batch, num_units] matrix
        """
        <your code here>
        return output

    def train(self):
        """
        Set layer into train mode (for DropOut, BatchNorm, etc)
        """
        pass

    def eval(self):
        """
        Set layer into evaluation mode
        """
        pass

In [ ]:
class Softmax:
    def __init__(self):
        self.params = []
        
    def forward(self, input):
        """
        Applies softmax to each row and then applies component-wise log
        Input shape: [batch, num_units]
        Output shape: [batch, num_units]
        """
        <your code here>
        return output

    def train(self):
        """
        Set layer into train mode (for DropOut, BatchNorm, etc)
        """
        pass

    def eval(self):
        """
        Set layer into evaluation mode
        """
        pass

In [ ]:
def crossentropy(activations, target):
    """
    returns negative log-likelihood of target under model represented by
    activations (log probabilities of classes)
    each arg has shape [batch, num_classes]
    output shape: 1 (scalar)
    """
    <your code here>
    return loss

In [ ]:
network = []
hidden_layers_size = 32
network.append(Dense(X_train.shape[1], hidden_layers_size))
network.append(ReLU())
network.append(Dense(hidden_layers_size, hidden_layers_size))
network.append(ReLU())
network.append(Dense(hidden_layers_size, 10))
network.append(Softmax())

In [ ]:
import torch.utils.data

In [ ]:
class SGDOptimizer:
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate

    def step(self, params, grads):
        """
        Take list of parameters and list of their gradients,
        make one optimization step for parameters in-place.
        """
        <your code here>

In [ ]:
def process_batch(batch_data, batch_target):
    """
    Make forward pass through network,
    compute gradients w. r. t. parameters,
    return loss.
    """
    <your code here>
    return loss    

In [ ]:
def run_epoch(dataset, prefix='Test loss:', optimizer=None):
    # Change mode for all layers.
    for layer in network:
        if optimizer:
            layer.train()
        else:
            layer.eval()

    batch_size = 100
    batchgenerator = torch.utils.data.DataLoader(dataset, batch_size, True)

    avg_loss = 0
    for i, (batch_data, batch_target) in enumerate(batchgenerator):
        avg_loss += (process_batch(batch_data, batch_target) - avg_loss) / (i + 1)
        if optimizer:
            optimizer.step(
                [param.data for layer in network for param in layer.params],
                [param.grad.data for layer in network for param in layer.params]
            )
            for layer in network:
                for param in layer.params:
                    param.grad.data.zero_()
    print(prefix, avg_loss, flush=True)

In [ ]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

sgd = SGDOptimizer(0.5)

num_epochs = 20
for i in range(num_epochs):
    run_epoch(train_dataset, 'Train loss:', sgd)
    run_epoch(test_dataset,  'Test loss:',  None)

## Различные модификации/улучшения

In [ ]:
class RMSPropOptimizer:
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate

    def step(self, params, grads):
        """
        Take list of parameters and list of their gradients,
        make one optimization step for parameters in-place.
        """
        <your code here>

In [ ]:
class AdamOptimizer:
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate

    def step(self, params, grads):
        """
        Take list of parameters and list of their gradients,
        make one optimization step for parameters in-place.
        """
        <your code here>

In [ ]:
class DenseWithDropOut:
    def __init__(self, input_units, output_units, dropout_rate):
        """
        A dense layer is a layer which performs a learned
        affine transformation and applies dropout:
        f(x) = (W x + b) o Bernoulli(p, size=output_units)
        """
        self.dropout_rate = dropout_rate
        self.train_mode = True
        # initialize weights with small random numbers from normal distribution
        self.weights = <your code here>
        self.biases = <your code here>
        self.params = [self.weights, self.biases]
        
    def forward(self, input):
        """
        Perform an affine transformation with dropout.
        In training mode:
        f(x) = (W x + b) o Bernoulli(p, size=output_units)
        In evaluation mode:
        f(x) = (W x + b) p 
        
        input shape: [batch, input_units]
        output shape: [batch, output units]
        """
        <your code here>
        return output

    def train(self):
        """
        Set layer into train mode (for DropOut, BatchNorm, etc)
        """
        self.train_mode = True

    def eval(self):
        """
        Set layer into evaluation mode
        """
        self.train_mode = False

In [ ]:
class BatchNormLayer:
    def __init__(self, input_units, output_units, dropout_rate):
        """
        TBD
        """
        self.train_mode = True
        <your code here>
        self.params = [<your code here>]
        
    def forward(self, input):
        """
        TBD
        """
        <your code here>
        return output

    def train(self):
        """
        Set layer into train mode (for DropOut, BatchNorm, etc)
        """
        self.train_mode = True

    def eval(self):
        """
        Set layer into evaluation mode
        """
        self.train_mode = False

Проверяем, верно ли, что полносвязная сеть с dropout работает быстрее, чем обычная полносвязная сеть,
засчет динамического графа вычислений.

In [ ]:
network1 = [DenseWithDropOut(1000, 1000, 0.9), Dense(1000, 1)]
network2 = [Dense(1000, 1000), Dense(1000, 1)]
X = torch.randn(100, 1000)
# check whether DenseWithDropOut works faster than Dense
<your code here>

Видим, что достаточно глубокая сеть не обучается.

Изучаем влияние различных методов регуляризации и оптимизации на обучение.

In [ ]:
network = []
hidden_layers_size = 32
network.append(Dense(X_train.shape[1], hidden_layers_size))
network.append(ReLU())
for i in range(10):
    network.append(Dense(hidden_layers_size, hidden_layers_size))
    network.append(ReLU())
network.append(Dense(hidden_layers_size, 10))
network.append(Softmax())

In [ ]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

sgd = SGDOptimizer(0.5)

num_epochs = 20
for i in range(num_epochs):
    run_epoch(train_dataset, 'Train loss:', sgd)
    run_epoch(test_dataset,  'Test loss:',  None)

In [ ]:
<your code here>